In [1]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [6]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [7]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [8]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.3),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.2),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True,dropout=0.1),merge_mode='ave'),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 128)          164864    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 64)           98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 32)           24832     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,291,816
Trainable params: 8,291,816
Non-trainable params: 0
______________________________________________

In [9]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 56s 174ms/step - loss: 1.9138 - acc: 0.2185 - val_loss: 1.7572 - val_acc: 0.2538
Epoch 2/30
323/323 [==============================] - 53s 165ms/step - loss: 1.8060 - acc: 0.2509 - val_loss: 1.8256 - val_acc: 0.2597
Epoch 3/30
323/323 [==============================] - 53s 165ms/step - loss: 1.7295 - acc: 0.2874 - val_loss: 1.7667 - val_acc: 0.2862
Epoch 4/30
323/323 [==============================] - 53s 166ms/step - loss: 1.6985 - acc: 0.3018 - val_loss: 1.6829 - val_acc: 0.3812
Epoch 5/30
323/323 [==============================] - 53s 165ms/step - loss: 1.6052 - acc: 0.3603 - val_loss: 1.6906 - val_acc: 0.2879
Epoch 6/30
323/323 [==============================] - 53s 165ms/step - loss: 1.5717 - acc: 0.3630 - val_loss: 1.5867 - val_acc: 0.3904
Epoch 7/30
323/323 [==============================] - 53s 165ms/step - loss: 1.3396 - acc: 0.4585 - val_loss: 1.2362 - val_acc: 0.5131
Epoch 8/30
323/323 [==============================] - 5

In [10]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 6400)
(75271, 8)
(16524, 1, 6400)
(16524, 8)


In [28]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(2048,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),input_shape=(X_train.shape[1],X_train.shape[2]),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.2),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True,dropout=0.2,recurrent_dropout=0.1),merge_mode='ave'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True),merge_mode='ave'),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [29]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_18 (Bidirectio (None, 1, 2048)           138428416 
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 1, 1024)           25174016  
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 1, 512)            6295552   
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 1, 256)            1574912   
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 1, 64)             164352    
_________________________________________________________________
lstm_27 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

In [30]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/25
2117/2117 [==============================] - 182s 86ms/step - loss: 0.3963 - acc: 0.8707 - val_loss: 0.1865 - val_acc: 0.9598
Epoch 2/25
2117/2117 [==============================] - 180s 85ms/step - loss: 0.1709 - acc: 0.9622 - val_loss: 0.1384 - val_acc: 0.9710
Epoch 3/25
2117/2117 [==============================] - 180s 85ms/step - loss: 0.1589 - acc: 0.9656 - val_loss: 0.1344 - val_acc: 0.9714
Epoch 4/25
2117/2117 [==============================] - 181s 85ms/step - loss: 0.1526 - acc: 0.9663 - val_loss: 0.1402 - val_acc: 0.9692
Epoch 5/25
2117/2117 [==============================] - 179s 85ms/step - loss: 0.1477 - acc: 0.9673 - val_loss: 0.1344 - val_acc: 0.9714
Epoch 6/25
2117/2117 [==============================] - 180s 85ms/step - loss: 0.1421 - acc: 0.9690 - val_loss: 0.1353 - val_acc: 0.9684
Epoch 7/25
2117/2117 [==============================] - 180s 85ms/step - loss: 0.1399 - acc: 0.9698 - val_loss: 0.1337 - val_acc: 0.9681
Epoch 8/25
2117/2117 [===================

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [31]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 8s 15ms/step - loss: 0.1292 - acc: 0.9707
[0.12918736040592194, 0.9707092642784119]


In [0]:
y_pred=model.predict(X_test)

In [33]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [34]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

97.12544263427361
97.10217494187543
97.10214112449003
